In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def poblacion_inactiva_sin_ingresos(df, tipo='Aglomerado', base='Individual', confidence_level=0.95, pool=False):

    # En este caso dividir el indicador segun algun percentil de ingresos. Por ejemplo de 90 a 60. Ver literatura al respecto 

    """
    INPUTS
    df: DataFrame. Tabla input EPH
    tipo: string. Tipo de encuesta de la EPH, Aglomerado o Urbano. Default Aglomerado
    base: string. Tipo de base de la encuesta de la EPH, Individual u Hogar. Default Individual

    OUTPUTS
    ratio: DataFrame. Tabla con Ratios en tasa M/V de población inactiva que no estudia y no tiene ingresos propios, desagregado por Aglomerado o Provincia
    error: DataFrame. Tabla con los errores asociados a los Ratios

    """


    if tipo == 'Aglomerado':
        var = 'AGLOMERADO'
    elif tipo == 'Urbano':
        var = 'PROVINCIA'

    df_temp = df.query('CH06 >= 14')
    
    if pool:
        pondera = 'PONDII_new'
    else:
        pondera = 'PONDII'


    numerador = df_temp[(df_temp['ESTADO']==3) & (df_temp['P47T']==0) & (df_temp['CAT_INAC']!=3)].groupby(['CH04', var])[pondera].sum().unstack(level=0)
    denominador = df_temp.groupby(['CH04', var])[pondera].sum().unstack(level=0)

    tasa = numerador.div(denominador, fill_value=np.nan)
    tasa.columns = ['Varon', 'Mujer']

    size = df_temp[(df_temp['ESTADO']==3) & (df_temp['P47T']==0) & (df_temp['CAT_INAC']!=3)].groupby(['CH04', var]).size().to_frame().unstack(level=0)
    size.columns = ['N_v', 'N_m']

    p1 = tasa['Varon']
    p2 = tasa['Mujer']
    n1 = size['N_v']
    n2 = size['N_m']
    ratio = (p1 / p2).to_frame()
    ratio.rename(columns={0: 'Dependencia'}, inplace=True)
    
    # Calculate the standard error of the proportion ratio
    standard_error = np.sqrt(((1 / n1) * (p1 * (1 - p1))) + ((1 / n2) * (p2 * (1 - p2))))

    z = stats.norm.ppf((1 + confidence_level) / 2)
    margin_of_error = z * standard_error
    lower_bound = p1/p2 - margin_of_error
    upper_bound = p1/p2 + margin_of_error

    relative_standard_error = margin_of_error/(p1/p2)
    error = pd.concat([lower_bound, upper_bound, margin_of_error, relative_standard_error], axis=1)
    error.columns = ['LI', 'LS', 'ME', 'ER']

    return tasa*100, size, ratio*100, error*100
